## Data Extraction and Transformation from Meeting Minute PDFs

This is used to extract motion language from meeting minute PDFs to find each instance of club or organization funding from UCSB's Associated Students Finance and Business Committee. The process includes discarding irrelevant motions, normalization of non-uniform organization names, and loading of details into a singular CSV for use on Power BI.

PDFs we are extracting from are UCSB AS F&B Meeting Minutes, publicly available at [AS F&B Committee Minutes](https://asfb.as.ucsb.edu/minutes2018-2019/)

In [1]:
import re
import logging
from pathlib import Path

import pandas as pd
import pdfplumber
import rapidfuzz

In [2]:
# Ignore non-critical warnings from pdfminer through pdfplumber
logging.getLogger("pdfminer").setLevel(logging.ERROR)

# Open file of all organizations as a list
with open("orgs-ucsb.txt", "r") as all_orgs_text:
    all_orgs = [line.strip() for line in all_orgs_text]

In [3]:
# This function will convert pdf pages to text holding relevant motions
# (motions to fund will only be found after the action items header)
def text_from_pdf(pdf_path):

    collecting = False
    out = ''
    
    with pdfplumber.open(pdf_path) as pdf:
         
        for page in pdf.pages: 
            
            text = page.extract_text()

            if not collecting:

                if "action items" in text.lower():
                    collecting = True
                    
            if collecting:
                out += text
                
        return out
        

# Looks for and returns list of motions found in the text
def find_motions(text):
    
    pattern = r"motion language:(.*?)action: passed"
    motions = re.findall(pattern, text.replace("\n", " ").lower(), flags=re.DOTALL)
    
    return motions
    

# Parses motions, takes normalized club name and dollar amount
def find_motion_details(motions, date):

    motion_details_dict = []
    
    for motion in motions:
        
        pattern = r"motion to\s+(.*?)\s*\$\s*([\d,]+(?:\.\d{2})?)"

        details = re.findall(pattern, motion.lower())

        if not details or any(word in motion for word in ["affirm", "forward", "table", "allocate", "deny"]):
            continue

        raw_org_name, amount = details[0]

        # Cleans most of the words that confuse fuzzy matching
        cleaner_org_name = re.sub(r"\b( ucsb|fully fund|partially fund|strike| at|motion|fund| to| of|requesting| for)\b", "",
                              raw_org_name, flags=re.IGNORECASE).strip()
        # A motion to strike means we will want to undo an existing funding motion 
        if 'strike' in motion.lower():

            amount = '-' + amount

        _, accuracy, org_index = rapidfuzz.process.extract(cleaner_org_name, [org.lower() for org in all_orgs],
                                            scorer=rapidfuzz.fuzz.ratio,
                                            limit = 1)[0]

        motion_details_dict.append({'Club/Org Name' : all_orgs[org_index],
                                   'Non-normalized Name' : cleaner_org_name,
                                   'Normalization Accuracy' : accuracy,
                                   'Funding' : amount,
                                   'Date' : date
                                  })
        
    return(motion_details_dict)

In [4]:
# Folder of pdfs, from UCSB AS F&B Meeting Minutes, publicly available, see above
pdf_folder = "meeting-mins-pdfs"

all_motions_together = []

for file in Path(pdf_folder).iterdir():

    date_of_motion = re.sub(r"\b(meeting-mins-pdfs\\|Finance Committee Meeting Minutes.pdf|)\b", "",str(file).strip())
   
    if file.is_file():
        
        text = text_from_pdf(str(file))
        motions = find_motions(text)
        details = find_motion_details(motions, date_of_motion.strip())

        all_motions_together.extend(details)

df = pd.DataFrame(all_motions_together)

pd.concat([df.head(10), df.tail(10)])


,Club/Org Name,Non-normalized Name,Normalization Accuracy,Funding,Date
0,Girl Up,girl up,100.000000,900,01.06.2025
1,Girl Up,girl up,100.000000,550,01.06.2025
2,Model United Nations,model united nations,100.000000,"2,255",01.06.2025
3,Model United Nations,model united nations,100.000000,"5,44.24",01.06.2025
4,Staff,msa,50.000000,265,01.06.2025
5,UCSBreakin',ucsbreakin’,90.909091,"1,000",01.06.2025
6,Association for Computing Machinery,association computing machinery,93.939394,150,01.06.2025
7,Men's Rugby,men’s rugby club,74.074074,"16,071",01.06.2025
8,REALITY,reality,100.000000,345.07,01.06.2025
9,Gaucho Pep Band,gaucho pep band,100.000000,"9,474",01.06.2025


### Filtering out inaccurately normalized names
Fuzzy match has some trouble with acronyms and sometimes other issues with the earlier parsing. This displays values which have below a certain level of normalization accuracy so that we can manually fix them. This will show up, from least to greatest, the names that have below 90 accuracy.

In [5]:
filtered = df[df['Normalization Accuracy'] <= 90]

pd.set_option('display.max_rows', None)

filtered.sort_values(by="Normalization Accuracy", inplace=False)

,Club/Org Name,Non-normalized Name,Normalization Accuracy,Funding,Date
164,180 Degrees Consulting,,0.000000,"18,802.38",02.24.2025
70,Best Buddies,udip,37.500000,150.19,02.03.2025
226,FROUG,bfom,44.444444,300,10.14.2024
103,PUENTE,mun,44.444444,"2,060.05",02.10.2025
289,Alive,ieee,44.444444,"2,327",11.25.2024
262,Alive,ieee,44.444444,"5,500,",11.04.2024
270,FROUG,bfom,44.444444,"1,550",11.04.2024
42,One Love,move,50.000000,272.04,01.13.2025
162,Esports,evpsa,50.000000,"37,390",02.24.2025
296,Staff,msa,50.000000,"1,500",11.25.2024


In [6]:
df.loc[164, 'Club/Org Name'] = 'Mold Kits IVTU'
df.loc[70 , 'Club/Org Name'] = 'Undergraduate Diversity and Inclusion in Physics'
df.loc[226, 'Club/Org Name'] = 'BFOM A Capella'
df.loc[270, 'Club/Org Name'] = 'BFOM A Capella'
df.loc[103, 'Club/Org Name'] = 'Model United Nations'
df.loc[262, 'Club/Org Name'] = 'Institute of Electrical and Electronics Engineers'
df.loc[289, 'Club/Org Name'] = 'Institute of Electrical and Electronics Engineers'
df.loc[42 , 'Club/Org Name'] = 'Move@UCSB'
df.loc[241, 'Club/Org Name'] = 'VocalMotion'
df.loc[162, 'Club/Org Name'] = 'AS External Vice President for Statewide Affairs'
df.loc[4  , 'Club/Org Name'] = 'Muslim Student Association'
df.loc[296, 'Club/Org Name'] = 'Muslim Student Association'
df.loc[168, 'Club/Org Name'] = 'Women in Computer Science'
df.loc[240, 'Club/Org Name'] = None
df.loc[286, 'Club/Org Name'] = 'AS External Vice President for Statewide Affairs'
df.loc[43 , 'Club/Org Name'] = 'AS External Vice President for Statewide Affairs'
df.loc[298, 'Club/Org Name'] = 'Chemistry Club'
df.loc[68 , 'Club/Org Name'] = 'Model United Nations'
df.loc[157, 'Club/Org Name'] = 'Horizons Incentive Program'
df.loc[297, 'Club/Org Name'] = 'Horizons Incentive Program'
df.loc[59 , 'Club/Org Name'] = 'Horizons Incentive Program'
df.loc[35 , 'Club/Org Name'] = 'BFOM A Capella'
df.loc[112, 'Club/Org Name'] = None
df.loc[247, 'Club/Org Name'] = None
df.loc[126, 'Club/Org Name'] = None
df.loc[51 , 'Club/Org Name'] = 'American Institute of Chemical Engineers'
df.loc[127, 'Club/Org Name'] = 'BFOM A Capella'
df.loc[149, 'Club/Org Name'] = 'BFOM A Capella'
df.loc[128, 'Club/Org Name'] = 'AS Rent Support Program'
df.loc[288, 'Club/Org Name'] = 'Dance Team'
df.loc[301, 'Club/Org Name'] = 'Gaucho Creative Marketing Group'
df.loc[120, 'Club/Org Name'] = 'Gaucho Creative Marketing Group'
df.loc[79 , 'Club/Org Name'] = 'Collegiate Chapter of SAE International AKA Gaucho Racing'
df.loc[254, 'Club/Org Name'] = None
df.loc[158, 'Club/Org Name'] = 'Environmental Leadership Incubator'
df.loc[147, 'Club/Org Name'] = 'Santa Barbara Case Club'
df.loc[124, 'Club/Org Name'] = 'Club Tennis'
df.loc[39 , 'Club/Org Name'] = 'Comunidad Latinx Graduacion'
df.loc[163, 'Club/Org Name'] = 'Collegiate Chapter of SAE International AKA Gaucho Racing'

df

,Club/Org Name,Non-normalized Name,Normalization Accuracy,Funding,Date
0,Girl Up,girl up,100.000000,900,01.06.2025
1,Girl Up,girl up,100.000000,550,01.06.2025
2,Model United Nations,model united nations,100.000000,"2,255",01.06.2025
3,Model United Nations,model united nations,100.000000,"5,44.24",01.06.2025
4,Muslim Student Association,msa,50.000000,265,01.06.2025
5,UCSBreakin',ucsbreakin’,90.909091,"1,000",01.06.2025
6,Association for Computing Machinery,association computing machinery,93.939394,150,01.06.2025
7,Men's Rugby,men’s rugby club,74.074074,"16,071",01.06.2025
8,REALITY,reality,100.000000,345.07,01.06.2025
9,Gaucho Pep Band,gaucho pep band,100.000000,"9,474",01.06.2025


### Preparing final data
Now we want to put all the data we have collected into a CSV for use on Power BI or later analysis if needed. We will exclude now irrelevant information such as the non-normalized names and the normalization accuracy. 

In [7]:
final_df = df.drop(columns=['Non-normalized Name', 'Normalization Accuracy'])

final_df.to_csv('org-funding-data.csv', index=False)

final_df

,Club/Org Name,Funding,Date
0,Girl Up,900,01.06.2025
1,Girl Up,550,01.06.2025
2,Model United Nations,"2,255",01.06.2025
3,Model United Nations,"5,44.24",01.06.2025
4,Muslim Student Association,265,01.06.2025
5,UCSBreakin',"1,000",01.06.2025
6,Association for Computing Machinery,150,01.06.2025
7,Men's Rugby,"16,071",01.06.2025
8,REALITY,345.07,01.06.2025
9,Gaucho Pep Band,"9,474",01.06.2025
